In [1]:
from importnb import Notebook
with Notebook():
    import cleaning as cd
import config_options as cfg
import pandas as pd

In [2]:
data_base_merged=cd.data_base_merged

In [3]:
f_stock = (cfg.ddm_trading_stocks_pct/100)*(1+cfg.iva_pct)
f_opt   = (cfg.ddm_trading_opts_pct/100)*(1+cfg.iva_pct)
f_ex    = (cfg.ddm_exercising_options_pct/100)*(1+cfg.iva_pct)

In [4]:
MULT=int(cfg.multiplicador)

LEVELS = (1, 2, 3)

OPTION_PRICE_PER_SHARE = True 
OPT_SCALE = MULT if OPTION_PRICE_PER_SHARE else 1
COOLDOWN_SECONDS = cfg.COOLDOWN_SECONDS  

def _get(row, col):
    try:
        return row[col]
    except KeyError:
        return np.nan

def build_option_ladder(row, levels=LEVELS):
    ladder = []
    for L in levels:
        p = _get(row, f'of_{L}_precio')
        q = _get(row, f'of_{L}_size')
        if pd.notna(p) and pd.notna(q) and p > 0 and q > 0:
            # p * (1+f_opt) * MULT porque prima está por acción
            p_eff = p * (1 + f_opt) * OPT_SCALE  
            ladder.append((float(p_eff), int(q)))
    return ladder

def build_under_ladder(row, side, levels=LEVELS):
    ladder = []
    if side == 'call':
        for L in levels:
            p = _get(row, f'bi_{L}_precio_under')
            q = _get(row, f'bi_{L}_size_under')
            if pd.notna(p) and pd.notna(q) and p > 0 and q > 0:
                ladder.append((float(p) * (1 - f_stock), int(q)))
    else:
        for L in levels:
            p = _get(row, f'of_{L}_precio_under')
            q = _get(row, f'of_{L}_size_under')
            if pd.notna(p) and pd.notna(q) and p > 0 and q > 0:
                ladder.append((float(p) * (1 + f_stock), int(q)))
    return ladder

def match_tranches(opt_ladder, und_ladder, side, strike):
    i = j = 0
    shares_carry = 0
    traded_contracts = 0

    sum_p_opt = 0.0
    sum_p_und_sh = 0.0
    total_rev = 0.0
    total_cost = 0.0
    legs = []

    while i < len(opt_ladder) and j < len(und_ladder):
        p_opt, q_opt = opt_ladder[i]
        p_und, q_sh  = und_ladder[j]

        avail_und_contracts = (shares_carry + q_sh) // MULT
        if avail_und_contracts <= 0:
            shares_carry += q_sh
            j += 1
            continue

        size = int(min(q_opt, avail_und_contracts))
        if size <= 0:
            i += 1
            continue

        if side == 'call':
            rev_pc  = p_und * MULT
            cost_pc = p_opt + strike * MULT * (1 + f_ex)
        else:
            rev_pc  = strike * MULT * (1 - f_ex)
            cost_pc = p_opt + p_und * MULT

        pnl_pc = rev_pc - cost_pc
        if pnl_pc <= 0:
            break

        traded_contracts += size
        sum_p_opt += p_opt * size
        sum_p_und_sh += p_und * size
        total_rev  += rev_pc  * size
        total_cost += cost_pc * size
        legs.append((size, p_opt / MULT, p_und, pnl_pc / MULT))


        q_opt -= size
        opt_ladder[i] = (p_opt, q_opt)
        if q_opt <= 0:
            i += 1

        shares_needed = size * MULT
        if shares_carry >= shares_needed:
            shares_carry -= shares_needed
        else:
            shares_needed -= shares_carry
            shares_carry = 0
            q_sh -= shares_needed
            und_ladder[j] = (p_und, q_sh)
            if q_sh <= 0:
                j += 1

    if traded_contracts == 0:
        return {'contracts': 0}

    pnl_total = total_rev - total_cost
    return {
        'contracts': traded_contracts,
        'pnl_total': pnl_total,
        'pnl_per_contract': pnl_total / traded_contracts,
        'avg_p_opt_contract': sum_p_opt / traded_contracts,
        'avg_p_und_share':   sum_p_und_sh / traded_contracts,
        'legs': legs
    }
def simulate_arb(df, cooldown_by_ultimo=True):
    df = df.sort_values(['id_simbolo', 'biof_fecha']).copy()
    last_key = {}      # id_simbolo -> ultimo_fecha en el que ya se operó
    last_time = {}     # id_simbolo -> biof_fecha del último trade
    trades = []

    for idx, row in df.iterrows():
        sym  = row['id_simbolo']
        side = 'call' if str(row['instrument']).lower() == 'call' else 'put'
        strike = float(row['strike'])
        gate_key = row['ultimo_fecha'] if cooldown_by_ultimo else row['biof_fecha']
        t = pd.to_datetime(row['biof_fecha'])

        # chequeo cooldown por ultimo_fecha
        if cooldown_by_ultimo and pd.notna(gate_key) and (last_key.get(sym) == gate_key):
            continue

        # chequeo cooldown mínimo en segundos
        if sym in last_time:
            if (t - last_time[sym]).total_seconds() < COOLDOWN_SECONDS:
                continue

        opt_ladder = build_option_ladder(row)
        und_ladder = build_under_ladder(row, side)

        if not opt_ladder or not und_ladder:
            continue

        res = match_tranches(opt_ladder, und_ladder, side, strike)
        if res['contracts'] <= 0:
            continue

        trades.append({
            'biof_fecha': row['biof_fecha'],
            'ultimo_fecha': row['ultimo_fecha'],
            'id_simbolo': sym,
            'instrument': side,
            'strike': strike,
            'contracts': res['contracts'],

            'avg_p_opt_share': res['avg_p_opt_contract'] / MULT,
            'avg_p_und_share': res['avg_p_und_share'],
            'pnl_per_share': res['pnl_per_contract'] / MULT,
            'pnl_per_contract': res['pnl_per_contract'],
            'pnl_total': res['pnl_total'],
            'legs': res['legs'],
        })

        if cooldown_by_ultimo and pd.notna(gate_key):
            last_key[sym] = gate_key
        last_time[sym] = t

    trade_log = pd.DataFrame(trades)
    return trade_log

In [5]:
trade_log = simulate_arb(data_base_merged, cooldown_by_ultimo=True)

In [9]:
pnl=sum(trade_log["pnl_total"])
if pnl != 0:
    print(pnl)
else: 
    None

1625333.1448700796
